In [5]:
import os
from collections import namedtuple

from alpaca.data.requests import StockBarsRequest
from alpaca.data.timeframe import TimeFrame
from alpaca.data.historical import StockHistoricalDataClient
from alpaca.trading.client import TradingClient

In [7]:
AlpacaCreds = namedtuple('AlpacaCreds', ['api_key', 'secret_key'])
creds = AlpacaCreds(os.getenv('ALPACA_API_KEY'), os.getenv('ALPACA_SECRET_KEY'))

In [16]:
creds.api_key

'AK2EBTEKIKRJTNDI23DK'

In [12]:
trade_client = TradingClient(api_key=creds.api_key, secret_key=creds.secret_key)
stock_data_client = StockHistoricalDataClient(api_key=creds.api_key, secret_key=creds.secret_key)

In [14]:
trade_client.get_account()

APIError: {"code":40110000,"message":"request is not authorized"}

In [ ]:
request_params = StockBarsRequest(
  symbol_or_symbols=["BTC/USD"],
  timeframe=TimeFrame.Day,
  start="2022-09-01",
  end="2022-09-07"
)